PROBLEM STATEMENT

1. Load a text summarization model from hugging face (A new model input here).
2. Figure out HF_Token.
3. Have a gradio interface for me to give input and see output for this model.
4. I need export functionality, dark/light theme.

### Implementation Overview and Explanation

This notebook sets up a web-based text summarization application using Gradio, powered by a Hugging Face pre-trained BART model. Below is a breakdown of the key components:

1.  **Library Imports and Setup**
    *   `gradio as gr`: Used for building the interactive web interface.
    *   `transformers`: Provides classes like `AutoTokenizer` and `AutoModelForSeq2SeqLM` to load and use pre-trained models from Hugging Face.
    *   `torch`: The underlying deep learning library; used to check for GPU availability.
    *   `datetime`, `os`: Utilities for creating unique filenames for exported summaries.
    *   **Installation**: `!pip install -q transformers torch gradio accelerate` ensures all necessary libraries are installed.
    *   **Gradio Version Check**: `print(f"Gradio Version: {gr.__version__}")` confirms the installed Gradio version.

2.  **Model Loading**
    *   `model_name = "facebook/bart-large-cnn"`: Specifies the pre-trained BART model for summarization.
    *   **Device Check**: The code automatically detects if a GPU (`cuda`) is available and uses it; otherwise, it falls back to the CPU. This optimizes performance.
    *   `AutoTokenizer.from_pretrained(model_name)`: Loads the tokenizer specific to the BART model, which is essential for converting text into numerical inputs the model understands.
    *   `AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)`: Loads the pre-trained BART model itself and moves it to the detected device (GPU/CPU).

3.  **`summarize_text` Function**
    *   This is the core function that performs the summarization.
    *   It takes `text`, `max_length`, and `min_length` as input.
    *   **Input Handling**: Checks for empty input text.
    *   **Tokenization**: `tokenizer([text], max_length=1024, return_tensors='pt', truncation=True).to(device)` prepares the input text by tokenizing it and ensuring it fits the model's maximum input length.
    *   **Summary Generation**: `model.generate(...)` uses the loaded BART model to create a summary. Key parameters include:
        *   `num_beams=4`: Uses beam search for better summary quality.
        *   `max_length`, `min_length`: Controls the length of the generated summary.
        *   `early_stopping=True`: Stops generation when a complete summary is formed.
    *   **Decoding**: `tokenizer.decode(...)` converts the generated token IDs back into human-readable text.
    *   **Export Functionality**: The summary is saved to a temporary text file with a unique filename, which is then provided as a downloadable file in the Gradio interface.
    *   **Error Handling**: Includes a `try-except` block to catch and report any errors during summarization.

4.  **Gradio Interface (`gr.Blocks`)**
    *   `gr.Blocks` is used for more flexible and custom layouts compared to `gr.Interface`.
    *   **Theme**: `theme=gr.themes.Soft(primary_hue="blue")` sets a soft blue theme for the application.
    *   **Layout**: `gr.Row()` and `gr.Column()` are used to arrange elements in a structured way.
    *   **Header**: Displays the app title and a theme toggle button.
    *   **Input Area**: `gr.Textbox` for users to paste text, and `gr.Slider` widgets to control `max_length` and `min_length` of the summary.
    *   **Summarize Button**: Triggers the `summarize_text` function.
    *   **Output Area**: `gr.Textbox` to display the generated summary and `gr.File` for downloading the summary.
    *   **Theme Toggle**: A `gr.Button` with custom JavaScript (`js=...`) to switch between light and dark modes dynamically.
    *   **Event Handling**: `summarize_btn.click(...)` connects the button click event to the `summarize_text` function, passing inputs and receiving outputs.
    *   **Examples**: `gr.Examples` provides pre-filled text samples to demonstrate the app's functionality.
    *   **Markdown sections**: Include tips for usage and information about the model.

5.  **Custom CSS**
    *   `demo.css = """..."""`: Provides custom CSS rules specifically for the dark theme to ensure better visual consistency.

6.  **Launching the App**
    *   `demo.launch(share=True, debug=False, show_error=True)` starts the Gradio application.
    *   `share=True` generates a public shareable link, allowing others to access the app temporarily.

In [1]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # Changed from pipeline
import torch
from datetime import datetime
import os

# Install required packages
!pip install -q transformers torch gradio accelerate
print(f"Gradio Version: {gr.__version__}") # <--- ADDED: Check Gradio version

# Initialize the summarization model and tokenizer
print("🔄 Loading summarization model from Hugging Face...")
model_name = "facebook/bart-large-cnn"
print(f"Model: {model_name}")

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
print("✅ Model loaded successfully!")


# Global variable to track current theme
current_theme = "light"

def summarize_text(text, max_length, min_length):
    """Summarize the input text using the loaded model"""
    if not text or text.strip() == "":
        return "⚠️ Please enter some text to summarize.", None

    try:
        # Prepare the input for the model
        inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True).to(device)

        # Generate summary
        summary_ids = model.generate(
            inputs['input_ids'],
            num_beams=4,
            max_length=int(max_length),
            min_length=int(min_length),
            early_stopping=True
        )

        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        filename = f"/tmp/summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(summary_text)

        return summary_text, filename

    except Exception as e:
        return f"❌ Error: {str(e)}", None

with gr.Blocks(title="Text Summarization App", theme=gr.themes.Soft(primary_hue="blue")) as demo:

    # Header with theme toggle button
    with gr.Row():
        with gr.Column(scale=5):
            gr.Markdown(
                """
                # 📝 Text Summarization App
                ### Powered by Hugging Face's BART Model
                """
            )
        with gr.Column(scale=1, min_width=150):
            theme_state = gr.State(value="light")
            theme_btn = gr.Button("🌙 Switch to Dark Mode", size="sm", elem_id="theme-toggle")

    with gr.Row():
        with gr.Column(scale=3):
            input_text = gr.Textbox(
                label="📄 Input Text",
                placeholder="Paste your text here... (articles, documents, emails, etc.)",
                lines=12
            )

            with gr.Row():
                max_length_slider = gr.Slider(
                    minimum=50,
                    maximum=300,
                    value=130,
                    step=10,
                    label="📏 Max Summary Length (tokens)"
                )
                min_length_slider = gr.Slider(
                    minimum=10,
                    maximum=100,
                    value=30,
                    step=5,
                    label="📏 Min Summary Length (tokens)"
                )

            with gr.Row():
                summarize_btn = gr.Button("✨ Summarize", variant="primary", size="lg")
                # clear_btn = gr.Button("🗑️ Clear History", variant="secondary") # REMOVED

            summary_output = gr.Textbox(
                label="✅ Summary",
                lines=6,
                interactive=False
            )

        with gr.Column(scale=2): # This column will now be for current summary export only
            gr.Markdown("### 📥 Export Current Summary")
            # export_text = gr.Textbox(...) # REMOVED
            download_file = gr.File(
                label="💾 Download Current Summary",
                interactive=False
            )

    # Theme toggle function with JavaScript
    theme_btn.click(
        None,
        None,
        None,
        js="""
        () => {
            const html = document.querySelector('html');
            const button = document.getElementById('theme-toggle').querySelector('button');

            if (html.classList.contains('dark')) {
                html.classList.remove('dark');
                button.textContent = '🌙 Switch to Dark Mode';
                return 'light';
            } else {
                html.classList.add('dark');
                button.textContent = '☀️ Switch to Light Mode';
                return 'dark';
            }
        }
        """
    )

    # Event handlers
    summarize_btn.click(
        fn=summarize_text,
        inputs=[input_text, max_length_slider, min_length_slider],
        outputs=[summary_output, download_file]
    )

    # Examples
    gr.Examples(
        examples=[
            ["The quick brown fox jumps over the lazy dog. This is a simple sentence used to demonstrate text summarization. In reality, you would use much longer texts for meaningful summarization results. Machine learning models work best when given substantial input text to process and condense into meaningful summaries."],
            ["Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of intelligent agents. AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals."]
        ],
        inputs=input_text,
        label="📚 Example Texts"
    )

    gr.Markdown(
        """
        ### 💡 Tips:
        - 📝 Enter longer text (at least 100 words) for better results
        - 🚏 Adjust sliders to control summary length
        - 💾 Download button will appear with the current summary.
        - 🌟 Works best with: news articles, research papers, reports, emails
        - 🌓 Click the theme toggle button to switch between dark and light modes

        ### ℹ️ About the Model:
        - Model: `facebook/bart-large-cnn`
        - Source: Hugging Face Transformers
        - Optimized for English text summarization
        """
    )

# Add custom CSS for better dark mode support
demo.css = """
.dark {
    --body-background-fill: #0b0f19;
    --background-fill-primary: #111827;
    --background-fill-secondary: #1f2937;
    --border-color-primary: #374151;
}
"""

# Launch the app
print("\n🚀 Launching Gradio interface...")
print("=" * 80)

demo.launch(
    share=True,  # Creates a public shareable link
    debug=False,
    show_error=True
)

print("\n✅ App is now running!")
print("📱 A public link has been generated above for sharing")
print("🌓 Use the theme toggle button in the app to switch between dark and light modes!")

Gradio Version: 5.50.0
🔄 Loading summarization model from Hugging Face...
Model: facebook/bart-large-cnn
Using device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

✅ Model loaded successfully!


/tmp/ipython-input-2620495732.py:58: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Text Summarization App", theme=gr.themes.Soft(primary_hue="blue")) as demo:



🚀 Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91baf37236a344633e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ App is now running!
📱 A public link has been generated above for sharing
🌓 Use the theme toggle button in the app to switch between dark and light modes!
